In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels import api as sm
from statsmodels.graphics import tsaplots

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
import re

<p style="border:3px #ff0000  solid;">
Задача: спрогнозировать число пассажиров авиакомпании помесячно.

<p style="border:3px #ff0000  solid;">
1. Загрузите данные из источника по ссылке, ознакомьтесь с ними и подготовьте к аналитике.

In [ ]:
air_pass_df = pd.read_csv('C:/Users/Z/PycharmProjects/data_science_1t/task_2.6/AirPassengers.csv')
air_pass_df.columns = [re.sub('\\.+', '_', i).lower() for i in air_pass_df.columns]
air_pass_df

In [ ]:
air_pass_df.info()

     a) Найдите и удалите пропуски, если они есть.

In [ ]:
air_pass_df.isna().sum()

<p style="border:3px #00B344  solid;">
Пропуски отсутсвуют.
</p>

     b) Поменяйте тип столбца с временными отсечками в datetime.

In [ ]:
air_pass_df['month'] = pd.to_datetime(air_pass_df['month'], format="%Y-%m")

     c) Установите индексом данных столбец с временными отсечками и убедитесь, что он монотонно возрастает.

In [ ]:
air_pass_df.set_index('month', inplace=True)
air_pass_df.index.is_monotonic_increasing


<p style="border:3px #ff0000  solid;">
2. Проанализируйте временной ряд

     а) Оставьте для аналитики лишь с 1952 по 1957 год (включительно).

In [ ]:
air_pass_df = air_pass_df['1952':'1957']

    b) Реализуйте разложение на компоненты тренда и сезонности. Визуализируйте результат разложения и сделайте выводы.

    Есть ли тренд?

    Есть ли сезонность и какая она (ежемесячная/ежеквартальная/ежегодная и т.д.)?

    Является ли ряд стационарным и почему?

In [ ]:
decomposed = seasonal_decompose(air_pass_df['#passengers'])

fig, ax = plt.subplots(1, 1, figsize=[12, 5])
ax.plot(air_pass_df['#passengers'], c='indianred', label='Passengers')
ax.plot(decomposed.trend, c='darkslateblue', label='Trend')
ax.plot(decomposed.seasonal, c='forestgreen', label='Seasonal')
ax.plot(decomposed.resid, c='darkgoldenrod', label='Residual')
ax.set_ylabel('passengers')
fig.legend()

<p style="border:3px #00B344  solid;">
    1. Имеется положительный тренд, характеризующий ежегожный прирост количества пассажиров.
    2. Наблюдается ежегодная сезонность, приходящаяся на серидину года (летний период отпусков).
    3. Ряд не является стационарным, так как имеется положительный тренд (изменяется среднее количество пассажиров, перевезенных за год).
</p>

    c) Постройте график, чтобы оценить автокорреляцию и сделайте выводы

    У какого сдвига самая высокая автокорреляция?

    Подтверждаются ли выводы о наличии тренда графиком автокорреляции?

In [ ]:
acf = sm.tsa.acf(air_pass_df['#passengers'])
acf.shape

In [ ]:
fig = tsaplots.plot_acf(air_pass_df['#passengers'])
plt.show() 

<p style="border:3px #00B344  solid;">
    1. Из графика мы видим, что автокорреляция в среднем положительна, поэтому можно сделать вывод о наличии положительного тренда во временном ряду, это значит, что сделанные ранее выводы о наличии положительного тренда подтверждены.
    2. Самая высокая автокорреляция у первого сдвига, что говорит о его высокой статистической значимости.
</p>

<p style="border:3px #ff0000  solid;">
3. Подготовьте данные к обучению линейной модели

     a) Напишите функцию, которая будет генерировать новые признаки (календарные признаки, несколько колонок с разными сдвигами, несколько колонок с разными скользящими средними)

     Воспользуйтесь наработками из скринкастов.

     Добавьте аргумент, который будет отвечать за возможные сдвиги. Это должен быть список, по которому в цикле будет добавляться новая колонка lag_n, где n — число из этого списка, отвечающее за сдвиг.

     Добавьте аргумент, который будет отвечать за возможные размеры скользящего среднего. Это должен быть список, по которому в цикле будет добавляться новая колонка rolling_mean_m, где m — число из этого списка, отвечающее за размер окна, по которому вычисляется среднее. Не забудьте, что скользящее среднее нужно сдвигать, чтобы не было «заглядывания в будущее».

In [ ]:
def make_features(data, max_lag, rolling_mean_array):
    data['year'] = data.index.year
    data['month'] = data.index.month
    data['quarter'] = data.index.quarter
    
    for i in max_lag:
        data[f'lag_{i}'] = data['#passengers'].shift(i)
    
    for rolling_mean_size in rolling_mean_array:
        rolling_mean = data['#passengers'].rolling(rolling_mean_size).mean()
        data[f'rolling_mean_size_{rolling_mean_size}'] = rolling_mean.shift(1)
    
    return data

     b) Рассчитайте новые признаки с помощью реализованной функции, подав ей на вход произвольные параметры генерации новых признаков.

In [ ]:
air_pass_df = make_features(air_pass_df[['#passengers']].copy(), [3, 4, 8], [7, 9, 12])
air_pass_df.head()

    c) Удалите наблюдения с пропусками в новых переменных.

In [ ]:
air_pass_df.dropna(inplace=True)

    d) Разделите данные на признаки и целевую переменную.

    e) Разделите данные на обучающую и валидационную части. Не забудьте, что валидационная часть должна идти строго после обучающей.

In [ ]:
#Датафрейм air_pass_df передается в функцию train_test_split вместе со всеми признаками, в том числе и целевым!
#Чтобы валидационная часть шла строго после обучающей, укажем shuffle=False
train, test = train_test_split(air_pass_df, shuffle=False, test_size=0.2, random_state=123)

train_target = train['#passengers']
train_features = train.drop(['#passengers'], axis=1)
test_target = test['#passengers']
test_features = test.drop(['#passengers'], axis=1)

scaler = StandardScaler().fit(train_features)
train_features_norm = scaler.transform(train_features)
test_features_norm = scaler.transform(test_features)

<p style="border:3px #ff0000  solid;">
4. Обучите линейную модель машинного обучения и оцените ее качество 

    a) Обучите линейную модель на обучающей части данных.

In [ ]:
model = LinearRegression()
model.fit(train_features_norm, train_target)

    b) Рассчитайте значение метрики качества MAE на обучающей и валидационных частях. Если модель работает плохо (метрика выше 20), попробуйте выделить другой набор признаков, вызывая свою функцию с новыми параметрами.

In [ ]:
train_prediction = pd.Series(model.predict(train_features_norm))
train_prediction.index = train_features.index
test_prediction = pd.Series(model.predict(test_features_norm))
test_prediction.index = test_features.index

print("MAE обучающей выборки:" ,mean_absolute_error(train_target, train_prediction))
print("MAE тестовой выборки: " ,mean_absolute_error(test_target, test_prediction))

    c) Визуализируйте реальный временной ряд и прогнозные значения. 

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=[12, 5])

error = abs(test['#passengers'] - test_prediction)
error_mean = pd.Series([error.mean()] * error.shape[0])
error_mean.index = test.index

ax.plot(air_pass_df['#passengers'], c='indianred', label='Real data')
ax.plot(train_target, c='lightgrey', label='train')
ax.plot(train_prediction, c='darkslategrey', label='train_prediction', linestyle='--')
ax.plot(test_target, c='lightsteelblue', label='test')
ax.plot(test_prediction, c='darkslateblue', label='test_prediction', linestyle='--')
ax.plot(error, c='indianred', linestyle='--', label='prediction_error', linewidth=0.5)
ax.plot(error_mean, c='indianred', label='prediction_error')

fig.legend()

<p style="border:3px #00B344  solid;">
MAE обучающей выборки: 14.291535093036588
MAE тестовой выборки:  16.824070242963952
Полученных значений MAE удалось добиться после увеличения размера rolling_mean_size больше 5.
</p>

<p style="border:3px #ff0000  solid;">
5. Оцените качество модели на отложенной выборке. В качестве отложенной выборки у нас будут наблюдения с 1958 по 1960 год

    a) Возьмите из источника необходимый срез данных.

In [ ]:
air_pass_df_two = pd.read_csv('C:/Users/Z/PycharmProjects/data_science_1t/task_2.6/AirPassengers.csv')
air_pass_df_two.columns = [re.sub('\\.+', '_', i).lower() for i in air_pass_df_two.columns]

In [ ]:
air_pass_df_two['month'] = pd.to_datetime(air_pass_df_two['month'], format="%Y-%m")
air_pass_df_two.set_index('month', inplace=True)
air_pass_df_two.index.is_monotonic_increasing


In [ ]:
air_pass_df_two = air_pass_df_two['1957':'1960']

In [ ]:
air_pass_df_two.info()

In [ ]:
air_pass_df_two.isna().sum()

    b) Подготовьте данные к прогнозированию.

    Добавьте признаки с помощью реализованной ранее функции.

    Удалите наблюдения с пропусками.

    Если после удаления у вас осталось менее 36 наблюдений, откорректируйте срез из пункта 5.а.

    Отделите целевую переменную от признаков.

In [ ]:
air_pass_df_two = make_features(air_pass_df_two[['#passengers']].copy(), [3, 4, 8], [7, 9, 12])
air_pass_df_two.head()

In [ ]:
air_pass_df_two.dropna(inplace=True)

In [ ]:
#изначально выборка взята с учетом 1957 года, но из-за того, что rolling_mean_size_12 = 12, 
#после удаления пустых данных в выборке осталься период 1958-01-01 to 1960-12-01
air_pass_df_two.info()

In [ ]:
target_air_pass = air_pass_df_two['#passengers']
features_air_pass = air_pass_df_two.drop(['#passengers'], axis=1)

features_air_pass_norm = scaler.transform(features_air_pass)

    c) Постройте прогноз с помощью обученной ранее линейной модели.

In [ ]:
test_prediction_2 = pd.Series(model.predict(features_air_pass_norm))
test_prediction_2.index = features_air_pass.index

    d) Рассчитайте значение метрики качества на отложенной выборке.

In [ ]:
print("MAE отложенной выборки: " ,mean_absolute_error(target_air_pass, test_prediction_2))

    e) Визуализируйте реальный временной ряд и прогнозные значения.

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=[12, 5])

error = abs(air_pass_df_two['#passengers'] - test_prediction_2)
error_mean = pd.Series([error.mean()] * error.shape[0])
error_mean.index = target_air_pass.index

ax.plot(target_air_pass, c='indianred', label='Real data')
ax.plot(test_prediction_2, c='darkslateblue', label='test_prediction', linestyle='--')
ax.plot(error, c='indianred', linestyle='--', label='prediction_error', linewidth=0.5)
ax.plot(error_mean, c='indianred', label='prediction_error')

fig.legend()

<p style="border:3px #00B344  solid;">
Вывод: MAE отложенной выборки (34.57) находится в заданном пределе.
</p>